In [6]:
!! /Users/simongenin/anaconda/envs/ML/bin/pip install pulp

['Collecting pulp',
 '  Downloading PuLP-1.6.8.tar.gz (13.5MB)',
 'Requirement already satisfied: pyparsing>=2.0.1 in /Users/simongenin/anaconda/envs/ML/lib/python3.6/site-packages (from pulp)',
 'Building wheels for collected packages: pulp',
 '  Running setup.py bdist_wheel for pulp: started',
 "  Running setup.py bdist_wheel for pulp: finished with status 'done'",
 '  Stored in directory: /Users/simongenin/Library/Caches/pip/wheels/d0/73/a3/85621bee86fa3dc23ddc3e0b19d8dcf58e3bc232ccd05793ed',
 'Successfully built pulp',
 'Installing collected packages: pulp',
 'Successfully installed pulp-1.6.8']

In [9]:


# Bus Data

busData = {
    'LFA': {
        'cost' : 570000,
        'capacity' : 140
    },
    'LF':{
        'cost' : 390000,
        'capacity' : 92
    },
    'LE': {
        'cost' : 350000,
        'capacity' : 60
    }
}


# Cost function

numLFA : ???
numLF: ???
numLE: ???


In [6]:
### Cost Function


def costFunction(numLFA, numLF, numLE):
    return (numLFA * busData['LFA']['cost']) + (numLF * busData['LF']['cost']) + (numLE * busData['LE']['cost'])


In [8]:
### Initializing Data

data = {
    'timeslot_1': {
        'time': 240,
        
        'route_1' : {
            'passengers' : 700,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 30
        },
        
        'route_2' : {
            'passengers' : 2000,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 40
        }
        
    }
}

In [1]:
### Importing Modules

import pandas
import pulp

In [2]:
### Calculating max appearance from timeslot, route

def maxAppearanceLFA(route,timeslot,numLFA):
    timeslotDuration = data[timeslot]['time']
    roundtripDuration = data[timeslot][route]['duration']
    
    maxAppearLFA = numLFA*timeslotDuration/roundtripDuration
    return maxAppearLFA

def maxAppearanceLF(route,timeslot,numLF):
    timeslotDuration = data[timeslot]['time']
    roundtripDuration = data[timeslot][route]['duration']
    
    maxAppearLF = numLF*timeslotDuration/roundtripDuration
    return maxAppearLF

def maxAppearanceLE(route,timeslot,numLE):
    timeslotDuration = data[timeslot]['time']
    roundtripDuration = data[timeslot][route]['duration']
    
    maxAppearLE = numLE*timeslotDuration/roundtripDuration
    return maxAppearLE

In [10]:
### Linear Programming Optimization

def solveLP(route,timeslot):

    # Instantiate Problem
    model = pulp.LpProblem("Cost Optimisation", pulp.LpMinimize)

    # Number of buses of types
    numLFA = pulp.LpVariable('numLFA',lowBound = 0, cat = 'Integer')
    numLF = pulp.LpVariable('numLF',lowBound = 0, cat = 'Integer')
    numLE = pulp.LpVariable('numLE',lowBound = 0, cat = 'Integer')

    # Objective function
    model += costFunction(numLFA,numLF,numLE), "Cost"

    # Constraint 1 (Number of bus appearances)
    
    model += ((maxAppearanceLFA(route,timeslot,numLFA) + maxAppearanceLF(route,timeslot,numLF) + maxAppearanceLE(route,timeslot,numLE)) <= data[timeslot]['time']/data[timeslot][route]['bus_frequency'])
    
    # Constraint 2 (Passenger Capacity)

    model += (maxAppearanceLFA(route,timeslot,numLFA)*busData['LFA']['capacity']+
                maxAppearanceLF(route,timeslot,numLF)*busData['LF']['capacity']+
                maxAppearanceLE(route,timeslot,numLE)*busData['LE']['capacity']) <= data[timeslot][route]['passengers']
    
    
    # Solving
    
    model.solve()
    print('Status: ' + str(pulp.LpStatus[model.status]))
    
    print('numLFA: {}'.format(numLFA.varValue))
    print('numLF: {}'.format(numLF.varValue))
    print('numLE: {}'.format(numLE.varValue))
    
    print('Cost: ' + str(pulp.value(model.objective)))

In [11]:
solveLP('route_1', 'timeslot_1')

NameError: name 'numLF' is not defined